In [13]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import requests
import pickle
from os import listdir, path
from functools import partial
import re
from concurrent import futures
import time
from tqdm.notebook import tqdm

In [14]:
!ls

create_db.py	  food_data		     query_manager.py	sql_scripts
create_tables.py  populate_recipe_tables.py  reset_table.sh	utils.py
drop_tables.py	  __pycache__		     scrape_data.ipynb


In [15]:
raw_data_dir = "food_data"
listdir(raw_data_dir)

['interactions_test.csv',
 'PP_recipes.csv',
 'interactions_train.csv',
 'interactions_validation.csv',
 'RAW_recipes.csv',
 'RAW_interactions.csv',
 'PP_users.csv',
 'ingr_map.pkl']

In [16]:
create_file_path = partial(path.join, "food_data")
with open(create_file_path("ingr_map.pkl"), "rb") as f:
    ingredients_map_df= pickle.load(f)
ingredients_map_df.head()

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308


In [17]:
raw_recipe_df = pd.read_csv(create_file_path("RAW_recipes.csv"))
pp_recipe_df = pd.read_csv(create_file_path("PP_recipes.csv"))
raw_interaction_df = pd.read_csv(create_file_path("RAW_interactions.csv"))
pp_user_df = pd.read_csv(create_file_path("PP_users.csv"))

In [18]:
raw_recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [19]:
pp_recipe_df.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [20]:
pp_recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178265 entries, 0 to 178264
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 178265 non-null  int64 
 1   i                  178265 non-null  int64 
 2   name_tokens        178265 non-null  object
 3   ingredient_tokens  178265 non-null  object
 4   steps_tokens       178265 non-null  object
 5   techniques         178265 non-null  object
 6   calorie_level      178265 non-null  int64 
 7   ingredient_ids     178265 non-null  object
dtypes: int64(3), object(5)
memory usage: 10.9+ MB


In [21]:
raw_recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [22]:
filtered_raw_recipe_df = raw_recipe_df[raw_recipe_df.id.isin(pp_recipe_df.id)]

In [23]:
filtered_raw_recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9


In [24]:
def add_food_recipe_url(recipe_df):
    recipe_df["food_recipe_url"] = ("https://www.food.com/recipe/" 
                                    + recipe_df["name"].str.replace(" ", "-") 
                                    + "-" 
                                    + recipe_df["id"].astype(str)
                                   )
    return recipe_df

In [25]:
recipe_df = add_food_recipe_url(filtered_raw_recipe_df)

/u0/psvadrevu/miniconda3/envs/dl_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,food_recipe_url
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,https://www.food.com/recipe/arriba---baked-win...
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,https://www.food.com/recipe/a-bit-different--b...
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,https://www.food.com/recipe/alouette--potatoes...
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4,https://www.food.com/recipe/apple-a-day--milk-...
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9,https://www.food.com/recipe/aww--marinated-oli...


In [39]:
def scrape_recipe(food_recipe_url):
    result = requests.get(food_recipe_url)
    soup = BeautifulSoup(result.content, 'html.parser')
    quantities = []
    measurement_units = []
    for tag in soup.find_all("li", {"class": "recipe-ingredients__item"}):
        try:
            quantity = tag.find("div", {"class": "recipe-ingredients__ingredient-quantity"}).text.strip()
            quantity = re.sub(' +',' ',quantity)
            quantities.append(quantity)
            ingredient_part = tag.find("div", {"class": "recipe-ingredients__ingredient-parts"}).text.strip()
            ingredient_part = re.sub(' +',' ',ingredient_part)
            measurement_unit = (re.search("(\(.*\))* *[A-za-z\.]*", ingredient_part).group(0) 
                                if (quantity!="") else "")
            measurement_units.append(measurement_unit)
        except AttributeError:
            continue

#     if (len(quantities) != len(measurement_units)):
#         return tag
    

    return [quantities, measurement_units]
# TODO: Add strip and replace(" +", " ") later to vectorize it

In [38]:
scrape_recipe(recipe_df.iloc[0]["food_recipe_url"])

[['1', '1 -2', '1 -2', '', '', '', ''],
 ['lb', 'teaspoon', 'teaspoon', '', '', '', '']]

In [27]:
recipe_df.iloc[0]["food_recipe_url"]

'https://www.food.com/recipe/arriba---baked-winter-squash-mexican-style-137739'

In [246]:
%%time
for i in range(100):
    scrape_recipe(recipe_df.iloc[i]["food_recipe_url"])

CPU times: user 5.89 s, sys: 212 ms, total: 6.11 s
Wall time: 37.7 s


In [40]:
%%time
MAX_PROCESSES = 30
with futures.ProcessPoolExecutor(max_workers=MAX_PROCESSES) as executor:
    scraped_measurements = np.array(list(executor.map(scrape_recipe, recipe_df["food_recipe_url"].values)))

/u0/psvadrevu/miniconda3/envs/dl_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 1min 3s, sys: 11.5 s, total: 1min 15s
Wall time: 51min 36s


In [47]:
scraped_measurements.shape

(178265, 2)

In [51]:
scraped_measurements[:, 0]

array([list(['1', '1 -2', '1 -2', '', '', '', '']),
       list(['1', '1', '3', '1⁄2', '', '2']),
       list(['2', '12', '2', '2', '1', '5', '2', '1⁄8', '1⁄8', '', '']),
       ..., list(['1', '2', '1', '1', '1', '1']),
       list(['1', '2', '1', '1', '2', '1', '2', '2', '20', '4', '4']),
       list(['6', '1⁄4', '1', '2', '1 1⁄2', '1⁄4', '', '', ''])],
      dtype=object)

In [52]:
recipe_df["quantities"] = scraped_measurements[:, 0]
recipe_df["measurement_units"] = scraped_measurements[:, 1]

/u0/psvadrevu/miniconda3/envs/dl_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/u0/psvadrevu/miniconda3/envs/dl_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
with open("scraped_food_measurements.pkl", "wb") as f:
    pickle.dump(scraped_measurements, file=f)

In [59]:
recipe_df.to_csv("raw_178k_recipes_with_scraped_measurements.csv", index=False)

In [46]:
recipe_df.iloc[2]

name                                              alouette  potatoes
id                                                             59389
minutes                                                           45
contributor_id                                                 68585
submitted                                                 2003-04-14
tags               ['60-minutes-or-less', 'time-to-make', 'course...
nutrition                  [368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]
n_steps                                                           11
steps              ['place potatoes in a large pot of lightly sal...
description        this is a super easy, great tasting, make ahea...
ingredients        ['spreadable cheese with garlic and herbs', 'n...
n_ingredients                                                     11
food_recipe_url    https://www.food.com/recipe/alouette--potatoes...
Name: 3, dtype: object

In [257]:
pd.Series(['(16 ounce) package frozen broccoli cuts, thawed', 'tbsp. gamgren. eroe'])

array([list(['1', '1', '1 1⁄2', '1', '1', '1⁄2 - 3⁄4', '1⁄2', '1', '1 3⁄4']),
       list(['(16 ounce) package frozen broccoli cuts, thawed', '(10 3/4 ounce) can cream of chicken soup', 'cups shredded sharp cheddar cheese, divided', 'teaspoon garlic powder', 'teaspoon ground black pepper', 'teaspoon salt', 'cup milk', 'teaspoon soy sauce', 'cups French-fried onions, divided'])],
      dtype=object)

In [316]:
re.search('\(.*\) [A-za-z\.]*', "(16 ounce) package frozen broccoli cuts, thawed").group(0)
re.search("(\(.*\))* *[A-za-z\.]*", "tbsp. gamgren. eroe").group(0)

'tbsp.'

In [272]:
string = "(16 ounce) package frozen broccoli cuts, thawed"
string[string.find(")")+1:]
re.search("^([\w\-]+)", string)

In [210]:
np.array([[["1", "2"], ["this", "this"]], [["1", "2"], ["this", "this"]]])

array([[['1', '2'],
        ['this', 'this']],

       [['1', '2'],
        ['this', 'this']]], dtype='<U4')

In [198]:
%%time
MAX_THREADS = 10
with futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    numpy.array(executor.map(scrape_recipe, recipe_df["food_recipe_url"].iloc[:1000].values))

CPU times: user 1min 35s, sys: 5.57 s, total: 1min 41s
Wall time: 1min 15s
